In [1]:
import mwu_measures
import pandas as pd
import numpy as np
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher

mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
this_corpus = mwu_measures.processing_corpus.process_corpus('bnc', 'bnc_tokenized.txt', chunk_size=10000000, verbose=True)
this_corpus.create_totals()
ngram_selection = mwu_examples['Expression'].tolist()
bigrams = [ngram for ngram in ngram_selection if len(ngram.split()) == 2]
# ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)


# this_corpus = mwu_measures.processing_corpus.process_corpus(test_corpus=True)
# bigrams = ['b d', 'c b', 'a c']


88073 lines processed
178347 lines processed
251137 lines processed
331017 lines processed
417430 lines processed
515742 lines processed
613852 lines processed
688626 lines processed
771777 lines processed
859529 lines processed
942900 lines processed
1047131 lines processed
1123302 lines processed
1235027 lines processed
1333326 lines processed
1429373 lines processed
1524762 lines processed
1622597 lines processed
1725782 lines processed
1809093 lines processed
1884758 lines processed
1961270 lines processed
2040098 lines processed
2120332 lines processed
2197854 lines processed
2289980 lines processed
2373487 lines processed
2463569 lines processed
2537313 lines processed
2608775 lines processed
2690149 lines processed
2759435 lines processed
2861830 lines processed
2953964 lines processed
3046485 lines processed
3145655 lines processed
3259778 lines processed
3376309 lines processed
3475369 lines processed
3566620 lines processed
3654773 lines processed
3770351 lines processed
3855

In [2]:
bigrams

['.22 caliber',
 '.38 caliber',
 '1 kgs',
 '1 samuel',
 '1 timothy',
 '10-undecenoic acid',
 '18-karat gold',
 '1st class',
 '1st-class mail',
 '20/20 vision',
 '22-karat gold',
 '30 minutes',
 '365 days',
 '3d printer',
 '3d scanner',
 '4-3 suspension',
 '4-h club',
 '8 ball',
 '800 number',
 'a battery',
 'a bit',
 'a blowhard',
 'a bunch',
 'a capella',
 'a cappella',
 'a couple',
 'à deux',
 'a doddle',
 'a drive',
 'a few',
 'a fortiori',
 'a freak-out',
 'a further',
 'a getaway',
 'a go',
 'a goner',
 'a good',
 'a goof-off',
 'a handful',
 'a hang-up',
 'a heart-to-heart',
 'a heel',
 'a high-flyer',
 'a kickback',
 'à la',
 'a level',
 'a little',
 'a lot',
 'a major',
 'a mere',
 'a must',
 'a no-brainer',
 'a norange',
 'a one',
 'a pair',
 'a peach',
 'a pleasure',
 'a plus',
 'a posteriori',
 'a priori',
 'a propos',
 'a put-down',
 'a rip-off',
 'a roll',
 'a sec',
 'a shame',
 'a single',
 'a smokescreen',
 'a sport',
 'a stretch',
 'a student',
 'a to-do',
 'a ton',
 'a

In [3]:
helper = Fetcher(this_corpus.corpus_conn)

In [4]:
helper.create_bigram_query(bigrams)

In [5]:
helper.allocate_filtered_db()

Token Frequency

In [6]:
helper.conn.execute("""
    CREATE OR REPLACE TABLE token_freq AS
    SELECT 
        ug_1,
        ug_2,
        total_freq AS token_freq
    FROM filtered_db_total
    RIGHT JOIN this_query
    USING(ug_1, ug_2)
""")
helper("SELECT * FROM token_freq ORDER BY token_freq DESC").fetch_df()

,ug_1,ug_2,token_freq
0,of,a,124760.0
1,there,is,57158.0
2,the,same,55861.0
3,out,of,47463.0
4,have,to,42749.0
...,...,...,...
47481,x-ray,photography,NaN
47482,yellow,cake,NaN
47483,yellow,jack,NaN
47484,yeoman's,service,NaN


In [7]:
helper(
    """
    CREATE OR REPLACE TABLE ignore_ngrams AS
    SELECT * 
    FROM token_freq 
    WHERE token_freq IS NOT DISTINCT FROM NULL
""")
helper(
    """
    CREATE OR REPLACE TABLE reduced_query AS
    SELECT * 
    FROM this_query
    ANTI JOIN ignore_ngrams
    USING(ug_1, ug_2)
""")

In [8]:
helper(
    """
    CREATE OR REPLACE TABLE token_freq AS
    SELECT *
    FROM token_freq
    SEMI JOIN reduced_query
    USING(ug_1, ug_2)
""")

Type Frequencies

In [9]:
helper("""
    CREATE OR REPLACE TABLE type_1 AS
    SELECT 
        ug_2,
        COUNT( * ) AS typef_1
    FROM filtered_db_total
    WHERE ug_2 IN (SELECT ug_2 FROM reduced_query)
    GROUP BY
        ug_2
""")
helper("""
    CREATE OR REPLACE TABLE type_2 AS
    SELECT 
        ug_1,
        COUNT( * ) AS typef_2
    FROM filtered_db_total
    WHERE ug_1 IN (SELECT ug_1 FROM reduced_query)
    GROUP BY
        ug_1
""")

In [10]:
helper(
    """
    CREATE OR REPLACE TABLE type_freq AS
    WITH temp_1 AS (
        SELECT *
        FROM reduced_query
        LEFT JOIN type_1
        USING (ug_2)
    )
    SELECT *
    FROM temp_1
    LEFT JOIN type_2
    USING (ug_1)
""")
helper("SELECT * FROM type_freq ORDER BY typef_2 DESC").fetch_df()

,ug_1,ug_2,typef_1,typef_2
0,the,above,4524,150958
1,the,bends,136,150958
2,the,bomb,462,150958
3,the,burbs,1,150958
4,the,can,17937,150958
...,...,...,...,...
31768,uric,acid,827,1
31769,venae,cavae,1,1
31770,verd,antique,239,1
31771,wishy,washy,7,1


Dispersion

In [11]:
helper("""
CREATE OR REPLACE TABLE corpus_ngram_combs AS 
    SELECT corpus, ug_1, ug_2
    FROM (SELECT DISTINCT ug_1, ug_2 FROM filtered_db)
    SEMI JOIN reduced_query
    USING(ug_1, ug_2)
    CROSS JOIN (SELECT DISTINCT corpus FROM filtered_db)
""")

In [12]:
helper("""
    CREATE OR REPLACE TABLE disp_table AS
    SELECT corpus, ug_1, ug_2, COALESCE(freq, 0) as freq
    FROM corpus_ngram_combs
    LEFT JOIN filtered_db
    USING(corpus, ug_1, ug_2)
""")

In [13]:
helper(
    """
    CREATE OR REPLACE TABLE disp_table AS
    SELECT *
    FROM disp_table 
    LEFT JOIN corpus_proportions
    USING(corpus)
    WHERE ug_1 IN (SELECT ug_1 FROM this_query) AND ug_2 IN (SELECT ug_2 FROM this_query)
""")

In [14]:
helper(
    """
    CREATE OR REPLACE TABLE disp_table AS
    SELECT 
        corpus,
        disp_table.ug_1,
        disp_table.ug_2,
        freq / total_freq AS proportion,
        corpus_prop
    FROM disp_table
    LEFT JOIN filtered_db_total
    USING(ug_1, ug_2)
""")

In [15]:
helper(
    """
    CREATE OR REPLACE TABLE dispersion AS
    WITH kld_table AS (SELECT 
        corpus,
        ug_1,
        ug_2,
        CASE
            WHEN proportion > 0 THEN proportion * log2(proportion / corpus_prop)
            ELSE 0
        END AS KLD
    FROM disp_table
    )
    SELECT 
        ug_1,
        ug_2,
        1 - pow(EXP(1), -SUM(KLD)) as dispersion
    FROM kld_table
    GROUP BY (ug_1, ug_2)
    ORDER BY dispersion
""")

Associations @_@ Heavy use of CTE

In [16]:
helper(
    """
    CREATE OR REPLACE TABLE unigram_totals AS
    SELECT 
        ug,
        SUM(freq) AS freq,
        SUM(SUM(freq)) OVER () AS total_ug,
    FROM unigram_db
    GROUP BY (ug)
""")

In [17]:
helper(
    """
    CREATE OR REPLACE TABLE rel_freqs AS
    SELECT 
        ug_1,
        ug_2,
        token_freq,
        unigram_totals.freq AS ug_1_freq,
        unigram_totals.total_ug 
    FROM token_freq
    LEFT JOIN unigram_totals
    ON token_freq.ug_1 = unigram_totals.ug
    """)
helper(
    """
    CREATE OR REPLACE TABLE rel_freqs AS
    SELECT 
        ug_1,
        ug_2,
        token_freq,
        ug_1_freq,
        unigram_totals.freq AS ug_2_freq,
        rel_freqs.total_ug
    FROM rel_freqs
    LEFT JOIN unigram_totals
    ON rel_freqs.ug_2 = unigram_totals.ug
    """)

In [18]:
helper("""
    CREATE OR REPLACE TABLE associations AS
    WITH probs_db AS (
        SELECT
            ug_1,
            ug_2, 
            token_freq / ug_1_freq AS prob_2_1,
            token_freq / ug_2_freq AS prob_1_2,
            ug_1_freq / total_ug AS prob_1,
            ug_2_freq / total_ug AS prob_2
        FROM rel_freqs
    ), all_probs AS (
        SELECT
            ug_1,
            ug_2,
            prob_2_1,
            prob_1_2,
            prob_1,
            prob_2,
            1 - prob_2_1 AS prob_no_2_1,
            1 - prob_1_2 AS prob_no_1_2,
            1 - prob_1 AS prob_no_1,
            1 - prob_2 AS prob_no_2
        FROM probs_db
    ), forward_kld AS (
        SELECT
            ug_1,
            ug_2,
            prob_2_1 * log2(prob_2_1 / prob_2) AS kld_1,
            CASE
                WHEN prob_no_2_1 = 0 THEN 0
                ELSE (1 - prob_2_1) * log2((1 - prob_2_1) / (1 - prob_2))
            END AS kld_2
        FROM all_probs
    ), forward_assoc AS (
        SELECT
            ug_1,
            ug_2,
            1 - pow(EXP(1), -(kld_1 + kld_2)) AS fw_assoc
        FROM forward_kld
    ), backward_kld AS (
        SELECT
            ug_1,
            ug_2,
            prob_1_2 * log2(prob_1_2 / prob_1) AS kld_1,
            CASE
                WHEN prob_no_1_2 = 0 THEN 0
                ELSE (1 - prob_1_2) * log2((1 - prob_1_2) / (1 - prob_1))
            END AS kld_2
        FROM all_probs
    ), backward_assoc AS (
        SELECT
            ug_1,
            ug_2,
            1 - pow(EXP(1), -(kld_1 + kld_2)) AS bw_assoc
        FROM backward_kld
    )
    SELECT *
    FROM forward_assoc
    LEFT JOIN backward_assoc
    USING(ug_1, ug_2)
    ORDER BY bw_assoc DESC
""")

In [19]:
helper("SELECT * FROM associations").fetch_df()
# Now it's good

,ug_1,ug_2,fw_assoc,bw_assoc
0,venae,cavae,1.000000e+00,1.000000e+00
1,mirabile,dictu,9.999953e-01,1.000000e+00
2,higgledy,piggledy,1.000000e+00,1.000000e+00
3,hapax,legomenon,9.996402e-01,1.000000e+00
4,petitio,principii,9.999999e-01,1.000000e+00
...,...,...,...,...
31770,kill,for,7.087886e-09,3.563361e-11
31771,drive,in,7.757663e-09,3.449629e-11
31772,shy,from,1.038004e-08,2.792855e-11
31773,tool,case,1.668109e-10,8.026579e-12


Entropy differences

In [20]:
# Slot 2
# Real entropy
helper(
    """
    CREATE OR REPLACE TABLE entropy_real_2 AS
    WITH all_freqs AS (
        SELECT
            *
        FROM filtered_db_total
        WHERE ug_1 IN (SELECT ug_1 FROM reduced_query)
    ), ug_1_freqs AS (
        SELECT *
        FROM reduced_query
        LEFT JOIN all_freqs
        USING(ug_1)
    ), freqs AS (
        SELECT
            ug_1,
            ug_2 AS target,
            ug_2_1 AS ug_2,
            total_freq
        FROM ug_1_freqs
    ), totals AS (
        SELECT
            ug_1,
            SUM(total_freq) AS total
        FROM freqs
        GROUP BY ug_1
    ), probs AS (
        SELECT 
            ug_1,
            target,
            ug_2,
            total_freq / total AS prob
        FROM freqs
        LEFT JOIN totals
        USING(ug_1)
    ), info AS (
        SELECT
            ug_1,
            target,
            CASE
                WHEN prob > 0 THEN prob * log2(prob)
                ELSE 0
            END AS info
        FROM probs
    ), entropy AS (
        SELECT 
            ug_1,
            target AS ug_2,
            SUM(CASE WHEN info < 0 THEN -info ELSE 0 END) AS entropy,
            log2(COUNT (*)) AS normalizer
        FROM info
        GROUP BY ug_1, target
    )
    SELECT
        ug_1,
        ug_2,
        CASE
            WHEN normalizer > 0 THEN entropy / normalizer
            ELSE 0
        END as entropy_real
    FROM entropy
    ORDER BY entropy_real DESC
""")

# Counterfactual entropy

helper(
    """
    CREATE OR REPLACE TABLE entropy_cf_2 AS
    WITH all_freqs AS (
        SELECT
            *
        FROM filtered_db_total
        WHERE ug_1 IN (SELECT ug_1 FROM reduced_query)
    ), ug_1_freqs AS (
        SELECT *
        FROM reduced_query
        LEFT JOIN all_freqs
        USING(ug_1)
    ), freqs AS (
        SELECT
            ug_1,
            ug_2 AS target,
            ug_2_1 AS ug_2,
            CASE
                WHEN ug_2 = ug_2_1 THEN 0
                ELSE total_freq
            END AS total_freq
        FROM ug_1_freqs
    ), totals AS (
        SELECT
            ug_1,
            SUM(total_freq) AS total
        FROM freqs
        GROUP BY ug_1
    ), probs AS (
        SELECT 
            ug_1,
            target,
            ug_2,
            total_freq / total AS prob
        FROM freqs
        LEFT JOIN totals
        USING(ug_1)
    ), info AS (
        SELECT
            ug_1,
            target,
            CASE
                WHEN prob > 0 THEN prob * log2(prob)
                ELSE 0
            END AS info
        FROM probs
        WHERE target != ug_2
    ), entropy AS (
        SELECT 
            ug_1,
            target AS ug_2,
            SUM(CASE WHEN info < 0 THEN -info ELSE 0 END) AS entropy,
            log2(COUNT (*)) AS normalizer
        FROM info
        GROUP BY ug_1, target
    )
    SELECT
        ug_1,
        ug_2,
        CASE
            WHEN normalizer > 0 THEN entropy / normalizer
            ELSE 0
        END as entropy_cf
    FROM entropy
    ORDER BY entropy_cf DESC
""")
helper(
    """
    CREATE OR REPLACE TABLE entropy_2 AS
    SELECT 
        ug_1,
        ug_2,
        entropy_cf - entropy_real AS entropy_2
    FROM entropy_real_2
    INNER JOIN entropy_cf_2
    USING(ug_1, ug_2)
""")


In [21]:
# Slot 1
# Why is this stratospherically slower than slot 2?????? is it because "the"??
helper(
    """
    CREATE OR REPLACE TABLE entropy_real_1 AS
    WITH all_freqs AS (
        SELECT
            *
        FROM filtered_db_total
        WHERE ug_2 IN (SELECT ug_2 FROM reduced_query)
    ), ug_2_freqs AS (
        SELECT *
        FROM reduced_query
        LEFT JOIN all_freqs
        USING(ug_2)
    ), freqs AS (
        SELECT
            ug_2,
            ug_1 AS target,
            ug_1_1 AS ug_1,
            total_freq
        FROM ug_2_freqs
    ), totals AS (
        SELECT
            ug_2,
            SUM(total_freq) AS total
        FROM freqs
        GROUP BY ug_2
    ), probs AS (
        SELECT 
            ug_2,
            target,
            ug_1,
            total_freq / total AS prob
        FROM freqs
        LEFT JOIN totals
        USING(ug_2)
    ), info AS (
        SELECT
            ug_2,
            target,
            CASE
                WHEN prob > 0 THEN prob * log2(prob)
                ELSE 0
            END AS info
        FROM probs
    ), entropy AS (
        SELECT 
            ug_2,
            target AS ug_1,
            SUM(CASE WHEN info < 0 THEN -info ELSE 0 END) AS entropy,
            log2(COUNT (*)) AS normalizer
        FROM info
        GROUP BY ug_2, target
    )
    SELECT
        ug_1,
        ug_2,
        CASE
            WHEN normalizer > 0 THEN entropy / normalizer
            ELSE 0
        END as entropy_real
    FROM entropy
    ORDER BY entropy_real DESC
""")

helper(
    """
    CREATE OR REPLACE TABLE entropy_cf_1 AS
    WITH all_freqs AS (
        SELECT
            *
        FROM filtered_db_total
        WHERE ug_2 IN (SELECT ug_2 FROM reduced_query)
    ), ug_2_freqs AS (
        SELECT *
        FROM reduced_query
        LEFT JOIN all_freqs
        USING(ug_2)
    ), freqs AS (
        SELECT
            ug_2,
            ug_1 AS target,
            ug_1_1 AS ug_1,
            CASE
                WHEN ug_1 = ug_1_1 THEN 0
                ELSE total_freq
            END AS total_freq
        FROM ug_2_freqs
    ), totals AS (
        SELECT
            ug_2,
            SUM(total_freq) AS total
        FROM freqs
        GROUP BY ug_2
    ), probs AS (
        SELECT 
            ug_2,
            target,
            ug_1,
            total_freq / total AS prob
        FROM freqs
        LEFT JOIN totals
        USING(ug_2)
    ), info AS (
        SELECT
            ug_2,
            target,
            CASE
                WHEN prob > 0 THEN prob * log2(prob)
                ELSE 0
            END AS info
        FROM probs
        WHERE target != ug_1
    ), entropy AS (
        SELECT 
            ug_2,
            target AS ug_1,
            SUM(CASE WHEN info < 0 THEN -info ELSE 0 END) AS entropy,
            log2(COUNT (*)) AS normalizer
        FROM info
        GROUP BY ug_2, target
    )
    SELECT
        ug_1,
        ug_2,
        CASE
            WHEN normalizer > 0 THEN entropy / normalizer
            ELSE 0
        END as entropy_cf
    FROM entropy
    ORDER BY entropy_cf DESC
""")

helper(
    """
    CREATE OR REPLACE TABLE entropy_1 AS
    SELECT 
        ug_1,
        ug_2,
        entropy_cf - entropy_real AS entropy_1
    FROM entropy_real_1
    INNER JOIN entropy_cf_1
    USING(ug_1, ug_2)
""")

In [22]:
#now it's good
helper("""
       CREATE OR REPLACE TABLE entropy_diffs AS
       SELECT * FROM entropy_1
       INNER JOIN entropy_2
       USING(ug_1, ug_2)
       """)

In [23]:
helper("""
       CREATE OR REPLACE TABLE raw_measures AS
       SELECT 
              *,
              2 AS ngram_length
       FROM reduced_query
       INNER JOIN token_freq USING(ug_1, ug_2)
       INNER JOIN dispersion USING(ug_1, ug_2)
       INNER JOIN type_freq USING (ug_1, ug_2)
       INNER JOIN entropy_diffs USING (ug_1, ug_2)
       INNER JOIN associations USING(ug_1, ug_2)
       """)

In [24]:
helper("SELECT * FROM raw_measures").fetch_df()

,ug_1,ug_2,token_freq,dispersion,typef_1,typef_2,entropy_1,entropy_2,fw_assoc,bw_assoc,ngram_length
0,1st,class,16.0,0.732644,2005,270,0.000278,-0.000988,5.776097e-02,0.004185,2
1,a,battery,206.0,0.044579,313,71512,-0.014431,0.000034,1.545372e-04,0.242534,2
2,a,bit,16432.0,0.220602,1106,71512,-0.023982,-0.000101,2.565659e-02,0.915398,2
3,a,bunch,586.0,0.119691,225,71512,0.011449,0.000030,8.382538e-04,0.821681,2
4,a,capella,2.0,0.940355,22,71512,0.001896,0.000036,3.825570e-07,0.022242,2
...,...,...,...,...,...,...,...,...,...,...,...
31582,sans,serif,11.0,0.886609,10,52,0.023451,0.022138,8.054401e-01,0.989773,2
31583,sans,serif,11.0,0.886609,10,52,0.023451,0.022138,8.054401e-01,0.989773,2
31584,sans,serif,11.0,0.886609,10,52,0.023451,0.022138,8.054401e-01,0.989773,2
31585,sans,serif,11.0,0.886609,10,52,0.023451,0.022138,8.054401e-01,0.989773,2


In [31]:
entropy_limits = [-0.1, 0.1]
helper(
    f"""
    CREATE OR REPLACE TABLE normalized_measures AS
    SELECT
        ug_1,
        ug_2,
        (log(token_freq) - log(1)) / (log(ngram_totals.max_token) - log(1)) AS token_freq,
        1 - dispersion AS dispersion,
        (log(typef_1) - log(1)) / (log(ngram_totals.max_type1) - log(1)) AS type_1,
        (log(typef_2) - log(1)) / (log(ngram_totals.max_type2) - log(1)) AS type_2,
        CASE
            WHEN entropy_1 < {entropy_limits[0]} THEN 0
            WHEN entropy_1 > {entropy_limits[1]} THEN 1
            ELSE (entropy_1 - {entropy_limits[0]}) / ({entropy_limits[1]} - {entropy_limits[0]})
        END AS entropy_1,
        CASE
            WHEN entropy_2 < {entropy_limits[0]} THEN 0
            WHEN entropy_2 > {entropy_limits[1]} THEN 1
            ELSE (entropy_2 - {entropy_limits[0]}) / ({entropy_limits[1]} - {entropy_limits[0]})
        END AS entropy_2,
        fw_assoc,
        bw_assoc
    FROM raw_measures
    LEFT JOIN ngram_totals
    USING (ngram_length)
    """
    )

In [32]:
helper("SELECT * FROM normalized_measures").fetch_df()

,ug_1,ug_2,token_freq,dispersion,type_1,type_2,entropy_1,entropy_2,fw_assoc,bw_assoc
0,1st,class,0.204767,0.267356,0.645090,0.469479,0.501392,0.495062,5.776097e-02,0.004185
1,a,battery,0.393485,0.955421,0.487521,0.937346,0.427846,0.500169,1.545372e-04,0.242534
2,a,bit,0.716899,0.779398,0.594618,0.937346,0.380091,0.499493,2.565659e-02,0.915398
3,a,bunch,0.470695,0.880309,0.459515,0.937346,0.557245,0.500148,8.382538e-04,0.821681
4,a,capella,0.051192,0.059645,0.262251,0.937346,0.509478,0.500182,3.825570e-07,0.022242
...,...,...,...,...,...,...,...,...,...,...
31582,sans,serif,0.177094,0.113391,0.195357,0.331348,0.617256,0.610691,8.054401e-01,0.989773
31583,sans,serif,0.177094,0.113391,0.195357,0.331348,0.617256,0.610691,8.054401e-01,0.989773
31584,sans,serif,0.177094,0.113391,0.195357,0.331348,0.617256,0.610691,8.054401e-01,0.989773
31585,sans,serif,0.177094,0.113391,0.195357,0.331348,0.617256,0.610691,8.054401e-01,0.989773


In [33]:
weight_dict = {'token_freq': 1/4, 'dispersion': 1/4, 'type_1': 1/8, 'type_2': 1/8, 'entropy_1': 1/16, 'entropy_2': 1/16, 'fw_assoc': 1/16, 'bw_assoc': 1/16}

helper(
    f"""
    SELECT
        ug_1,
        ug_2,
        {weight_dict['token_freq']} * (token_freq) + 
        {weight_dict['dispersion']} * (dispersion) + 
        {weight_dict['type_1']} * type_1 + 
        {weight_dict['type_2']} * type_2 + 
        {weight_dict['entropy_1']} * entropy_1 + 
        {weight_dict['entropy_2']} * entropy_2 + 
        {weight_dict['fw_assoc']} * fw_assoc + 
        {weight_dict['bw_assoc']} * bw_assoc 
        AS MWU_score
    FROM normalized_measures
    ORDER BY MWU_score DESC
""").fetch_df()

,ug_1,ug_2,MWU_score
0,rather,than,0.804153
1,the,same,0.793573
2,more,than,0.778260
3,at,least,0.777343
4,the,whole,0.769021
...,...,...,...
31582,frontal,lobotomy,0.150271
31583,ride,horseback,0.150068
31584,citrus,medica,0.139243
31585,otitis,externa,0.136719


In [34]:
# TODO: function with (source, target) arguments and f strings
# TODO: batches to not crash memory when using complete corpus. Worth it? faster because of the simpler joins? worth it to hash?